Предполагается, что в рамках данного задания ученики сами будут изучать документацию, как минимум используя функционал `tab` и `shift+tab` от Jupyter

1. Импортируйте библиотеку pandas 
2. Считайте файл `wage.csv` в DataFrame с названием `wage`. Используйте для этого функцию pd.read_csv()

`person_id` - идентификатор человека

`gender` - пол, 0 - женский, 1 - мужской

`wage` - заработная плата в рублях

In [1]:
import pandas as pd

In [2]:
wage = pd.read_csv('wage.csv')
wage

,person_id,gender,wage
0,0,1,46793.603811
1,1,1,33481.575720
2,2,1,44523.699084
3,3,1,15995.576829
4,4,0,10282.631224
...,...,...,...
1001,995,1,66503.737185
1002,996,1,9972.956272
1003,997,0,104504.616392
1004,998,1,98927.903076


3. Поменяйте колонку `gender` так, чтобы были записи `F` (female) и `M` (male) вместо 0 и 1

In [3]:
wage['gender'] = wage['gender'].apply(lambda x: 'F' if x == 0 else 'M')

# or
# wage.loc[wage.gender == 0, 'gender'] = 'F' 
# wage.loc[wage.gender == 1, 'gender'] = 'M' 

# or
# wage['gender'] = wage.gender.apply(lambda x: {0: 'F', 1: 'M'}[x])


wage

,person_id,gender,wage
0,0,M,46793.603811
1,1,M,33481.575720
2,2,M,44523.699084
3,3,M,15995.576829
4,4,F,10282.631224
...,...,...,...
1001,995,M,66503.737185
1002,996,M,9972.956272
1003,997,F,104504.616392
1004,998,M,98927.903076


4. Давайте посчитаем среднюю зарплату у мужчин и женщин. Для этого используйте метод `groupby` со следующим синтаксисом:

`dataframe.groupby(Название_колонки_для_группировки)[Перечисление_колонок_для_агрегации].функция_агрегации()`

In [4]:
wage.groupby('gender')['wage'].mean()

gender
F    40855.747261
M    46815.944005
Name: wage, dtype: float64

5. Теперь взглянем внимательнее на данные и обнаружим, что некоторые люди попали в выборку несколько раз. 
    1. Найдите таких людей. Подсказка: `value_counts()`
    0. Убедитесь, что записи по ним с одинаковым `wage`. Возможно, тут вам пригодится функция агрегации `nunique()`, отображающая количество разных значений
    0. Избавьтесь от повторяющихся значений. Подсказка: `drop_duplicates()`

In [5]:
# сколько раз повторился каждый person_id
cnt = wage.person_id.value_counts()
cnt[cnt > 1]

17    2
16    2
15    2
14    2
13    2
12    2
Name: person_id, dtype: int64

In [6]:
wage.groupby('person_id').wage.nunique().max()

1

In [7]:
wage = wage.drop_duplicates('person_id')
wage

,person_id,gender,wage
0,0,M,46793.603811
1,1,M,33481.575720
2,2,M,44523.699084
3,3,M,15995.576829
4,4,F,10282.631224
...,...,...,...
1001,995,M,66503.737185
1002,996,M,9972.956272
1003,997,F,104504.616392
1004,998,M,98927.903076


6. Теперь посмотрим внимательнее на зарплаты
    1. Охарактеризуйте имеющиеся данные по зарплатам. Подсказка: `describe`
    1. Избавьтесь от бессмысленных значений

In [8]:
wage.wage.describe()

count      1000.000000
mean      43694.227404
std       55352.539343
min     -287418.645743
25%       14489.682367
50%       27309.529498
75%       52021.080258
max      755320.874132
Name: wage, dtype: float64

In [9]:
wage = wage[wage.wage >= 0]
wage

,person_id,gender,wage
0,0,M,46793.603811
1,1,M,33481.575720
2,2,M,44523.699084
3,3,M,15995.576829
4,4,F,10282.631224
...,...,...,...
1001,995,M,66503.737185
1002,996,M,9972.956272
1003,997,F,104504.616392
1004,998,M,98927.903076


7. Давайте теперь посмотрим на зарплату с учетом бонуса. Для этого нам понадобится таблица `bonus.csv`. Считайте ее в переменную `bonus`. Заметьте, что она сохранена немного в другом формате, и вам понадобится уточнить параметр `sep` - разделитель записей. Сравните текущий файл с предыдущим и попробуйте решить проблему

In [10]:
bonus = pd.read_csv('bonus.csv', sep=';')
bonus

,person_id,bonus
0,905,85059.638382
1,836,7703.346074
2,287,3120.269742
3,548,5347.987142
4,575,137257.490614
...,...,...
445,488,28102.252903
446,913,55549.347647
447,616,620397.407705
448,110,14086.067488


8. Чтобы посчитать итоговую зарплату, нам нужно по каждому человеку знать и оклад, и премию. Для этого надо будет соединить (сджойнить) таблицы по `person_id`. Используйте для этого функцию `pd.merge`. Помните, что параметр `how` должен быть `'outer'`, чтобы сохранить те записи, что есть только в одной таблице. Результат запишите в новый dataframe `df`

In [11]:
df = pd.merge(wage, bonus, on='person_id', how='outer')
df

,person_id,gender,wage,bonus
0,0,M,46793.603811,3.332934e+04
1,1,M,33481.575720,NaN
2,2,M,44523.699084,3.192912e+06
3,3,M,15995.576829,2.196858e+04
4,4,F,10282.631224,NaN
...,...,...,...,...
994,997,F,104504.616392,5.380978e+04
995,998,M,98927.903076,NaN
996,999,M,108107.141368,NaN
997,43,NaN,NaN,2.687784e+05


9. Наконец, давайте посчитаем итоговую зарплату
    1. Замените отсутствующие записи в колонке `bonus` нулями
    1. Уберите людей без `wage` - это те "плохие" записи, от которых мы избавлялись на предыдущих шагах
    1. Сделайте новую колонку `total`, которая будет равна 12 окладам и премии
    1. Посчитайте среднюю и медианную итоговую зарплату в разрезе по полу. Подсказка: вместо функции агрегации можно написать `.agg()` и перечислить внутри нужные агрегаты

In [12]:
df.loc[df.bonus.isna(), 'bonus'] = 0
df.head(2)

,person_id,gender,wage,bonus
0,0,M,46793.603811,33329.3441
1,1,M,33481.575720,0.0000


In [13]:
df = df[~df.wage.isna()]
df

,person_id,gender,wage,bonus
0,0,M,46793.603811,3.332934e+04
1,1,M,33481.575720,0.000000e+00
2,2,M,44523.699084,3.192912e+06
3,3,M,15995.576829,2.196858e+04
4,4,F,10282.631224,0.000000e+00
...,...,...,...,...
992,995,M,66503.737185,3.452137e+03
993,996,M,9972.956272,3.892790e+05
994,997,F,104504.616392,5.380978e+04
995,998,M,98927.903076,0.000000e+00


In [14]:
df.loc[:, 'total'] = df.wage * 12 + df.bonus
df.head(2)

<ipython-input-14-1ab06d3a168e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'total'] = df.wage * 12 + df.bonus


,person_id,gender,wage,bonus,total
0,0,M,46793.603811,33329.3441,594852.589830
1,1,M,33481.575720,0.0000,401778.908641


In [15]:
df.groupby('gender').wage.agg(['median', 'mean'])

,median,mean
gender,,
F,26721.001610,41353.986687
M,29006.633828,47081.694269


10. Сохраните `df` в файл, используя метод `to_csv()`. Не записывайте индексы

In [16]:
df.to_csv('result.csv', index=False)